In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re

In [2]:
def scrape_odds(prop_rows, method_1, method_2, method_3): #repeat_name = 0
    
    method_odds = [0, 0, 0]

    for row in prop_rows:

        if(row.find("th", text = method_1)):
            best_bet = row.find("span", class_ = "bestbet")
            method_odds[0] = best_bet.string
            continue

        elif(row.find("th", text = method_2)):
            best_bet = row.find("span", class_ = "bestbet")
            method_odds[1] = best_bet.string
            continue

        elif(row.find("th", text = method_3)):
            best_bet = row.find("span", class_ = "bestbet")
            method_odds[2] = best_bet.string
            continue  

    return method_odds
            
def scrapeResults(url):

    #Create the frame variables to store
    results_data = pd.DataFrame(columns = ["Event_name",
                                            "Date",
                                            "Fighter_1",
                                            "Fighter_1_Submission_odds",
                                            "Fighter_1_TKO/KO_Odds",
                                            "Fighter_1_Decision",
                                            "Fighter_2",
                                            "Fighter_2_Submission_odds",
                                            "Fighter_2_TKO/KO_Odds",
                                            "Fighter_2_Decision"])
    
    
    page = requests.get(url)  
    soup = BeautifulSoup(page.text, "html.parser")

    #step 1. Find the first two names to iterate over, create the win method strings for each fighter & update best bet if the string is in the row.
    try:
        
        odds_table = soup.find("div", class_ = "table-scroller")


        fighters = odds_table.find_all('span', class_ = "t-b-fcc")
        fighters_list = []
        fighter_last_name_list = []
        win_styles = [" wins by submission", " wins by TKO/KO", " wins by decision"]

        #collect all fighters
        for person in fighters:
            fighters_list.append(person.string)
            names = person.string.split(" ")
            if names[len(names) - 1] not in fighter_last_name_list:
                fighter_last_name_list.append(names[len(names) - 1])
            else:
                return results_data
            
            

        prop_rows = odds_table.findAll('tr', class_="pr")



        #add validation using a newly created dict that will count the number of times each name has appeared so far.
        #if a name repeats, then use findnext for the amount of times it repeats

        fighter_iterable = len(fighters_list)

        for fighter_num in range(0, len(fighters_list), 2):


            fighter1 = fighters_list[fighter_num]
            fighter1_lastname = fighter1.split(" ")[-1]
#             fighter1_lastname = " ".join(fighter1.split(" ")[-1])


            fighter2 = fighters_list[fighter_num + 1]
            fighter2_lastname = fighter2.split(" ")[-1]
#             fighter2_lastname = " ".join(fighter2.split(" ")[-1])

            fighter1_method_odds = scrape_odds(prop_rows, fighter1_lastname + win_styles[0], fighter1_lastname + win_styles[1], fighter1_lastname + win_styles[2])
            fighter2_method_odds = scrape_odds(prop_rows, fighter2_lastname + win_styles[0], fighter2_lastname + win_styles[1], fighter2_lastname + win_styles[2])


            results_data = results_data.append({"Event_name" : soup.find("div", class_ ="table-header").a.string,
                                                "Date" : soup.find("span", class_ ="table-header-date").string,
                                                "Fighter_1" : fighter1,
                                                "Fighter_1_Submission_odds" : fighter1_method_odds[0],
                                                "Fighter_1_TKO/KO_Odds" : fighter1_method_odds[1],
                                                "Fighter_1_Decision" : fighter1_method_odds[2],
                                                "Fighter_2" : fighter2,
                                                "Fighter_2_Submission_odds" : fighter2_method_odds[0],
                                                "Fighter_2_TKO/KO_Odds" : fighter2_method_odds[1],
                                                "Fighter_2_Decision" : fighter2_method_odds[2]},
                                                ignore_index = True)


        return(results_data.dropna())
    
    
    except:
        print("Fight URL: {} does not work".format(url)) 
        return results_data
    
  

In [11]:
#iterate over all UFC Fights 
combined_scraped_results = pd.DataFrame(columns = ["Event_name",
                                                   "Date",
                                                   "Fighter_1",
                                                   "Fighter_1_Submission_odds",
                                                   "Fighter_1_TKO/KO_Odds",
                                                   "Fighter_1_Decision",
                                                   "Fighter_2",
                                                   "Fighter_2_Submission_odds",
                                                   "Fighter_2_TKO/KO_Odds",
                                                   "Fighter_2_Decision"])
                                                

for i in range(150, 250):

    results = scrapeResults("https://www.bestfightodds.com/search?query=ufc_" + str(i))
    
#     print(results.head())
    
    time_to_sleep = random.randint(4, 11)
    print("waiting {} seconds...".format(time_to_sleep))
    time.sleep(time_to_sleep)
    
    print("going to {}, found, recovered results of shape: {}".format(("https://www.bestfightodds.com/search?query=ufc_" + str(i)), results.shape))
    combined_scraped_results = combined_scraped_results.append(results)
    
    
print(combined_scraped_results.shape)

combined_scraped_results.head()

waiting 10 seconds...
going to https://www.bestfightodds.com/search?query=ufc_150, found, recovered results of shape: (10, 10)
Fight URL: https://www.bestfightodds.com/search?query=ufc_151 does not work
waiting 5 seconds...
going to https://www.bestfightodds.com/search?query=ufc_151, found, recovered results of shape: (0, 10)
waiting 8 seconds...
going to https://www.bestfightodds.com/search?query=ufc_152, found, recovered results of shape: (12, 10)
waiting 5 seconds...
going to https://www.bestfightodds.com/search?query=ufc_153, found, recovered results of shape: (0, 10)
waiting 5 seconds...
going to https://www.bestfightodds.com/search?query=ufc_154, found, recovered results of shape: (12, 10)
waiting 5 seconds...
going to https://www.bestfightodds.com/search?query=ufc_155, found, recovered results of shape: (12, 10)
waiting 7 seconds...
going to https://www.bestfightodds.com/search?query=ufc_156, found, recovered results of shape: (11, 10)
waiting 8 seconds...
going to https://www.b

waiting 6 seconds...
going to https://www.bestfightodds.com/search?query=ufc_214, found, recovered results of shape: (12, 10)
waiting 9 seconds...
going to https://www.bestfightodds.com/search?query=ufc_215, found, recovered results of shape: (11, 10)
waiting 7 seconds...
going to https://www.bestfightodds.com/search?query=ufc_216, found, recovered results of shape: (11, 10)
waiting 7 seconds...
going to https://www.bestfightodds.com/search?query=ufc_217, found, recovered results of shape: (11, 10)
waiting 5 seconds...
going to https://www.bestfightodds.com/search?query=ufc_218, found, recovered results of shape: (0, 10)
waiting 7 seconds...
going to https://www.bestfightodds.com/search?query=ufc_219, found, recovered results of shape: (10, 10)
waiting 7 seconds...
going to https://www.bestfightodds.com/search?query=ufc_220, found, recovered results of shape: (11, 10)
waiting 8 seconds...
going to https://www.bestfightodds.com/search?query=ufc_221, found, recovered results of shape: (1

,Event_name,Date,Fighter_1,Fighter_1_Submission_odds,Fighter_1_TKO/KO_Odds,Fighter_1_Decision,Fighter_2,Fighter_2_Submission_odds,Fighter_2_TKO/KO_Odds,Fighter_2_Decision
0,UFC 150: Henderson vs. Edgar II,August 11th,Benson Henderson,+637,+500,+140,Frankie Edgar,+1780,+850,+295
1,UFC 150: Henderson vs. Edgar II,August 11th,Ed Herman,+1085,+475,+400,Jake Shields,+445,+1905,+110
2,UFC 150: Henderson vs. Edgar II,August 11th,Donald Cerrone,+105,+500,+323,Melvin Guillard,+4500,+435,+800
3,UFC 150: Henderson vs. Edgar II,August 11th,Justin Lawrence,+1303,+725,+275,Max Holloway,+850,+260,+185
4,UFC 150: Henderson vs. Edgar II,August 11th,Buddy Roberts,+1215,+750,+1800,Yushin Okami,+600,+200,+155


In [12]:
combined_scraped_results.to_csv("fight_odds.csv", index = False)